In [1]:
import cv2
import numpy as np
import os

In [2]:
# cap = cv2.VideoCapture('images//IMG_0189.mp4')

# while 1:
#     success, image = cap.read()
#     if success==False:
#         print(0)
#         break
#     # Меняем его цветовую палитру
#     gray_main_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
#     # Находим конутры и возращаем их
#     contours = find_contours_of_cards(gray_main_image)
    
#     # Нахождение название карты
#     cards_location = find_coordinates_of_cards(contours, gray_main_image)
    
#     # Рисование вокруг карты его название прямоугольника
#     result_image = draw_rectangle_aroud_cards(cards_location, image)

#     cv2.imshow('Isa', result_image)

#     if cv2.waitKey(1) and 0xFF == ord('q'):
#         break

In [3]:
# Функция для нахождения нужной карты.
def find_features(img1):
    correct_matches_dct = {}
    directory = 'images//data_project2'
    array = os.listdir(directory)[1:]
    for image in array:

        # Импортируем необходимое фото
        img2 = cv2.imread(directory+'//'+image, 0)

        # Для обнаружения ключевых точек используется алгоритм ORB
        # Инициализация алгоритма
        orb = cv2.SIFT_create()

        # Мы находим ключевые точки и дискрипторы(которые кодируют интересную информацию в ряд чисел)
        # для эталонной карты и для карты, которую мы вырезали из главного изображения.
        kp1, des1 = orb.detectAndCompute(img1, None)
        kp2, des2 = orb.detectAndCompute(img2, None)

        # Рисование ключевых точек на изображениях
        img1_with_keypoints = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        img2_with_keypoints = cv2.drawKeypoints(img2, kp2, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)


        
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
    
        
        
        correct_matches = []
        for m, n in matches:
            if m.distance < 0.75*n.distance:
                correct_matches.append([m])
                correct_matches_dct[image.split('.')[0]] = len(correct_matches)
                # Визуализация совпадений

        # if len(correct_matches)==0:
        #     continue
        # for_draw_matches = [val[0] for val in correct_matches]
        # img_matches = cv2.drawMatches(img1, kp1, img2, kp2, for_draw_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

        
        # img1_with_keypoints = cv2.resize(img1_with_keypoints,\
        #                                  (img1_with_keypoints.shape[1]//2,\
        #                                   img1_with_keypoints.shape[0]//2))
        # img2_with_keypoints = cv2.resize(img2_with_keypoints, \
        #                                 (img2_with_keypoints.shape[1]//2,\
        #                                 img2_with_keypoints.shape[0]//2))
        # img_matches = cv2.resize(img_matches,\
        #                          (img_matches.shape[1]//2,\
        #                          img_matches.shape[0]//2))

        
        # cv2.imshow('keypoints1', img1_with_keypoints)
        # cv2.imshow('keypoints2', img2_with_keypoints)
        # cv2.imshow('draw mathces', img_matches)
        # cv2.waitKey(0)

    # Соритируем так по количеству дитекторов вроде
    correct_matches_dct = dict(sorted(correct_matches_dct.items(), key=lambda item: item[1], reverse=True))

    # Возращаем нулевое, из индексса
    return list(correct_matches_dct.keys())[0]

In [4]:
# Нахождение название карты
# cards_location = find_coordinates_of_cards(contours, gray_main_image)

In [5]:
# Функция для нахождения контуров карт
def find_contours_of_cards(image):

    blurred = cv2.GaussianBlur(image, (3, 3), 0)
    T, thresh_img = cv2.threshold(blurred, 215, 255, 
                                  cv2.THRESH_BINARY)

    # Поиск контуров осуществляется с помощью метода findContours(), где в качестве первого
    # аргумента эта функция принимает изображение, вторым — это тип контуров, который мы хотим извлечь.
    # Я использую cv2.RETR_EXTERNAL для извлечения только внешних контуров. К примеру, для того,
    # чтобы извлечь все контуры используют cv2.RETR_LIST, а последний параметром мы указываем метод аппроксимации контура.
    # Мы используем cv2.CHAIN_APPROX_SIMPLE, указывая на то, что все лишние точки будут удалены, тем самым экономя память.
    # Например, если вы нашли контур прямой линии, то разве вам нужны все точки этой линии, чтобы представить эту линию? Нет,
    # нам нужны только две конечные точки этой линии. Это как раз то, что и делает cv2.CHAIN_APPROX_SIMPLE.
    cnts, hir = cv2.findContours(thresh_img, 
                                cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
    return cnts

In [6]:
# Данная функция принимает контуры, которые мы нашли в предыдущей функции, а также основное изображение в оттенках серого.
def find_coordinates_of_cards(cnts, image):
    # Словарь в котором будут: ключ - название. значений - координаты
    cards_coordinates = {}
    
    for i in range(0, len(cnts)):
        # boundingRect - берет коориднаты контура и выдает прямоугольник который описывает эти контуры.
        x, y, w, h = cv2.boundingRect(cnts[i]) 

        # Исключаем ненужные контуры, так как нам нужна контуры площадь которых не близка к нулю условно.
        if w > 20 and h > 30: 
            # Выбираем необходимый нам прямоугольник, в котором будет место нужной нам карты
            img_crop = image[y - 15:y + h + 15,x - 15:x + w + 15]

            # Наша функция find_features которая опираясь на ключевые точки, возвращает нам название вырезанной карты.
            cards_name = find_features(img_crop)# [2:]

            # После этого добавляем всё в словарь.
            cards_coordinates[cards_name] = (x - 15, y - 15, x + w + 15, y + h + 15)

    # Соотвественно возращаем словарь
    return cards_coordinates

In [7]:
# И вслед за этим рисуем прямоугольник вокруг карты с помощью функции
def draw_rectangle_aroud_cards(cards_coordinates, image):
    for key, value in cards_coordinates.items():
        rec = cv2.rectangle(image, (value[0], value[1]), 
                            (value[2], value[3]), 
                            (255, 255, 0), 2)
        cv2.putText(rec, key, (value[0], value[1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    0.5, (36, 255, 12), 1)
    # return image
    cv2.imshow('Image', image)
    cv2.waitKey(0)

In [ ]:
main_image = cv2.imread('images//more_cards1.jpg')

# Меняем его цветовую палитру
gray_main_image = cv2.cvtColor(main_image, cv2.COLOR_BGR2GRAY)

# Находим конутры и возращаем их
contours = find_contours_of_cards(gray_main_image)

# Нахождение название карты
cards_location = find_coordinates_of_cards(contours, gray_main_image)

# Рисование вокруг карты его название прямоугольника
draw_rectangle_aroud_cards(cards_location, main_image)

KeyboardInterrupt: 